In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 8
fig_height = 4
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie
  
  CairoMakie.activate!(type = string(fig_format))
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/home/diego/local_repos/AGEC652_2024/slides/lecture_4_4"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| include: false
using Pkg
Pkg.activate(".")
Pkg.instantiate()
#Pkg.add("JuMP")
#Pkg.add("Ipopt")

  Activating project at `~/local_repos/AGEC652_2024/slides/lecture_4_4`


In [3]:
using JuMP, Ipopt;

In [4]:
f(x_1,x_2) = -exp(-(x_1*x_2 - 3/2)^2 - (x_2-3/2)^2);

In [5]:
model = Model(Ipopt.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Ipopt

In [6]:
# This is relative tol. Default is 1e-8
set_optimizer_attribute(model, "tol", 1e-6) 

In [7]:
#| output: true
@variable(model, x_1 >=0)

x_1

In [8]:
#| output: true
@variable(model, x_2 >=0)

x_2

In [9]:
@objective(model, Min, f(x_1,x_2))

-(exp(-((x_1*x_2 - 1.5) ^ 2.0) - (x_2² - 3 x_2 + 2.25)))

In [10]:
print(model)

In [11]:
optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality c

In [12]:
set_silent(model);

In [13]:
#|include: false
optimize!(model);

In [14]:
unc_x_1 = value(x_1)

1.0000000202020707

In [15]:
unc_x_2 = value(x_2)

1.4999999999998486

In [16]:
unc_obj = objective_value(model)

-0.9999999999999987

In [17]:
#|include: false
optimize!(model);

In [18]:
unc_obj = objective_value(model)

-0.9999999999999987

In [19]:
#| output: asis
@constraint(model, -x_1 +x_2^2 == 0)

x_2² - x_1 = 0

In [20]:
#| output: asis
print(model)

In [21]:
#| output: asis
optimize!(model)

In [22]:
#| include: false
optimize!(model)

In [23]:
eqcon_x_1 = value(x_1)

149806.47565619063

In [24]:
eqcon_x_2 = value(x_2)

387.04841513199693

In [25]:
value(-x_1 + x_2^2) # We can evaluate expressions too

0.0

In [26]:
eqcon_obj = objective_value(model)

-0.0

In [27]:
model2 = Model(Ipopt.Optimizer);
@variable(model2, x_1 >=0);
@variable(model2, x_2 >=0);
@objective(model2, Min, f(x_1, x_2));
@constraint(model2, -x_1 + x_2^2 <= 0);
optimize!(model2);

This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:        2
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -1.1449605e-02 0.00e+00 1.03e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [28]:
ineqcon_x_1 = value(x_1)

1.3578043178154473

In [29]:
ineqcon_x_2 = value(x_2)

1.1652486056670581

In [30]:
ineqcon_obj = objective_value(model2)

-0.8879747414490442

In [31]:
model3 = Model(Ipopt.Optimizer);
@variable(model3, x_1 >=0);
@variable(model3, x_2 >=0);
@objective(model3, Min, f(x_1, x_2));
@constraint(model3, c1, -x_1 + x_2^2 <= 1.5);
optimize!(model3);

This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:        2
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        1

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -1.1449605e-02 0.00e+00 1.03e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [32]:
ineqcon2_obj = objective_value(model3)

-1.0

In [33]:
ineqcon2_x_1 = value(x_1)

1.0000000054547011

In [34]:
ineqcon2_x_2 = value(x_2)

1.4999999933331216